In [4]:
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import netCDF4 as nc
from scipy.interpolate import RegularGridInterpolator
import time


In [5]:
start_day, end_day = 1462, 1826
df_data = pd.read_pickle(f"/srv/scratch/z5297792/Chapter2/df_data_{start_day}_{end_day}.pkl")


In [ ]:
def tracking(df_data, start_ID, next_num, VORT_WEIGHT=15000, R_THRESH=30):

    df_data = df_data.copy()
    # Initialize IDs: set first day IDs equal to their Eddy values.
    min_day = df_data['Day'].min()
    df_data['ID'] = -1
    df_data.loc[df_data['Day'] == min_day, 'ID'] = df_data.loc[df_data['Day'] == min_day, 'Eddy']
    df_data['ID'] = df_data['ID'].astype('Int64')
    next_num = df_data['ID'].max() + 1

    R_df = pd.DataFrame(columns=['D_diff', 'W_diff', 'R'])

    # Loop through days starting from min_day+1 to max_day.
    for day in range(min_day + 1, df_data['Day'].max() + 1):
        pres_day = df_data[df_data['Day'] == day].copy()
        pres_day = pres_day[~np.isnan(pres_day['x0'])]
        for e_pres in pres_day['Eddy'].unique():
            pres_eddy = pres_day[pres_day['Eddy'] == e_pres].iloc[0]
            assigned = False
            # Look back up to 4 days.
            for delta in range(1, 5):
                candidate_day = day - delta
                if candidate_day < 0:
                    continue
                candidate_prev = df_data[df_data['Day'] == candidate_day].copy()
                candidate_prev = candidate_prev[~np.isnan(candidate_prev['x0'])]
                for e_prev in candidate_prev['Eddy'].unique():
                    prev_eddy = candidate_prev[candidate_prev['Eddy'] == e_prev].iloc[0]
                    R = np.sqrt(
                        (pres_eddy['x0'] - prev_eddy['x0'])**2 +
                        (pres_eddy['y0'] - prev_eddy['y0'])**2 +
                        VORT_WEIGHT * (pres_eddy['w'] - prev_eddy['w'])**2    # IF ESPRA DIDNT WORK USE NENCIOLI!!
                    )
                    D_diff = np.hypot(pres_eddy['x0'] - prev_eddy['x0'], pres_eddy['y0'] - prev_eddy['y0'])
                    W_diff = np.abs(pres_eddy['w'] - prev_eddy['w'])
                    R_df.loc[len(R_df)] = {'D_diff': D_diff, 'W_diff': W_diff, 'R': R}
                    if R < R_THRESH and (pres_eddy['Cyc'] == prev_eddy['Cyc']) and not (pres_day['ID'] == prev_eddy['ID']).any():
                        df_data.loc[(df_data['Day'] == day) & (df_data['Eddy'] == e_pres), 'ID'] = prev_eddy['ID']
                        assigned = True
                        break
                if assigned:
                    break
            if not assigned:
                df_data.loc[(df_data['Day'] == day) & (df_data['Eddy'] == e_pres), 'ID'] = next_num
                next_num += 1

        if day % 20 == 0:
            print(day)
                
    return df_data, R_df

tic = time.perf_counter()

df_data, R_df = tracking(df_data)

toc = time.perf_counter()
print(f"Elapsed time: {toc - tic:.4f} seconds")

1480
1500
1520
1540
1560
1580
1600


In [ ]:
df_eddies = df_data[df_data['ID']>=0].copy()
col_to_move = 'ID'
new_order = [col_to_move] + [col for col in df_eddies.columns if col != col_to_move]
df_eddies = df_eddies[new_order]
df_eddies = df_eddies.sort_values(by=['ID', 'Day']).drop('Eddy', axis=1)
df_eddies = df_eddies[df_eddies.groupby('ID')['ID'].transform('count') >= 21]
df_eddies['ID'] = df_eddies['ID'].rank(method='dense').astype(int)
df_eddies = df_eddies.reset_index(drop=True)
df_eddies

In [ ]:
df_eddies.to_pickle(f"/srv/scratch/z5297792/Chapter2/df_eddies_{start_day}_{end_day}.pkl")
R_df.to_pickle(f"/srv/scratch/z5297792/Chapter2/R_df{start_day}_{end_day}.pkl")